In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

In [12]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction import text
extra = []
extra = [e.lower() for e in extra]
stop_words = text.ENGLISH_STOP_WORDS.union(extra)

def preprocessing_text(texto):
    # texto es un conjunto de 1 o más sentences. Lo divido por sentences.
    sent_texto = nltk.sent_tokenize(texto) 
    # cada sentence de texto, la tokenizo en palabras y le saco las palabras de menos de 3 letras
    # y las stop_words
    sent_texto_filtered = []
    for sent in sent_texto:
        words_sent = word_tokenize(sent)
        #words_sent_filtered = [w for w in words_sent if not w in stop_words]
        words_sent_filtered = [w.lower() for w in words_sent if (len(w) >= 3) and (w.lower() not in stop_words)]
        # reconstruyo la sentence con las words filtradas
        sent_texto_filtered.append(' '.join(words_sent_filtered))
    return sent_texto_filtered 

In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/arturo.torrestey/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [14]:
# inicializo el SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

def vader_sent_analyzer(texto):
    # calculo el compound ratio promedio de la sentences que se encuentran en texto
    # texto viene como una lista de sentences
    pos_val = []
    neg_val = []
    neu_val = []
    for sent in texto:
        ss = sid.polarity_scores(sent)
        pos_val.append(ss['pos'])
        neg_val.append(ss['neg'])
        neu_val.append(ss['neu'])
        
    return (np.mean(pos_val), np.mean(neg_val), np.mean(neu_val))

In [17]:
def apply_vader(train=True):
    '''
    input = archivo de textos con sentences de tweets y reddit
    output = dataframe con cantidad de textos pos, neg y neutros agrupados por fecha
    '''
    
    # armo el nombre del archivo a leer de los parametros
    dir_name = '../data'    
    #file_name = 'reddit+twitter_train_filter.csv' if train else 'reddit+twitter_test_filter.csv'
    file_name = 'tweets_train.csv' if train else 'tweets_test.csv'
          
    filename  = os.path.join(dir_name, file_name)
    
    # leo archivo y le cambio los nombres a las columnas
    tweets = pd.read_csv(filename, usecols=[1,2])
        
    # preproceso los datos de la columna texto y defino el vader compound ratio
    tweets['tuplas'] = [vader_sent_analyzer(preprocessing_text(str(x))) for x in tweets.text]
    
    tweets['pos'] = tweets['tuplas'].apply(lambda x: x[0])
    tweets['neg'] = tweets['tuplas'].apply(lambda x: x[1])
    tweets['neu'] = tweets['tuplas'].apply(lambda x: x[2])
    
    # selecciono las columnas para trabajar
    tweets = tweets.loc[:, ['date', 'pos', 'neg', 'neu']]
    
    # agrupo tweets por fecha, sumarizando la cantidad de pos, neg y neutros
    grouped = tweets.groupby('date').mean()
    
    # retorno df con resultados
    return grouped

In [18]:
sent_train = apply_vader(train=True)
sent_test  = apply_vader(train=False)

### Armado de datasets finales

In [19]:
prices_train = pd.read_csv('../data/prices_train.csv', index_col=0)
prices_test = pd.read_csv('../data/prices_test.csv', index_col=0)

In [20]:
trainset = pd.merge(sent_train, prices_train, on='date', how='left')
testset = pd.merge(sent_test, prices_test, on='date', how='left')

In [21]:
trainset.to_csv('../data/trainset_va.csv')
testset.to_csv('../data/testset_va.csv')